# Extrapolation Data Generation

This notebook generates 6-qubit quantum circuit datasets for evaluating extrapolation performance of models trained on smaller circuits (2–5 qubits). It constructs both Class A (variational ansatz) and Class B (QAOA-like) circuits under noiseless and hardware-calibrated noisy conditions. Each circuit is converted into a graph-structured data object with engineered node and global features, including gate type, qubit encoding, gate parameters, noise calibrations, and Laplacian positional embeddings (`k=8`).The datasets are saved in a PyTorch Geometric-compatible format and will be used for zero-shot and few-shot extrapolation tests using both Graph Neural Networks and Convolutional Neural Networks.

In [1]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Library Imports
import os
import numpy as np
import random
import torch
from torch_geometric.data import Data
from qiskit import QuantumCircuit, transpile
from qiskit.converters import circuit_to_dag
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel
from scipy.sparse.linalg import eigsh
import datetime
import time
import json

## Global Seeding

In [3]:
def set_global_seeds(seed: int):
    """
    Set global random seeds for reproducibility.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    print(f"Global seeds set to {seed} (random, numpy, torch)")

# Set default seed
set_global_seeds(42)

Global seeds set to 42 (random, numpy, torch)


## Directory Utilities

In [4]:
def ensure_dataset_dir(n_qubits, noise_type, circuit_class=None, root='../datasets'):
    """
    Ensure output directory exists.
    If circuit_class is None, create up to noise_type level (for calibration snapshot).
    """
    parts = [root, f"{n_qubits}-qubit", noise_type]
    if circuit_class:
        parts.append(circuit_class)
    folder = os.path.join(*parts)
    os.makedirs(folder, exist_ok=True)
    return folder


def save_dataset(data_list, n_qubits, noise_type, circuit_class, root='../datasets'):
    """
    Save the data_list (list of PyG Data) to disk in the correct directory.
    """
    folder = ensure_dataset_dir(n_qubits, noise_type, circuit_class, root=root)
    fname = f"dataset_{n_qubits}q_{noise_type}_{circuit_class}.pt"
    save_path = os.path.join(folder, fname)
    torch.save(data_list, save_path)
    print(f"Saved dataset ({len(data_list)} samples) to {save_path}")

## Gate Encoding and Constants

In [5]:
GATE_TYPES = ['rx', 'ry', 'rz', 'h', 'x', 'y', 'z', 'cx']
GATE_TYPE_IDX = {g: i for i, g in enumerate(GATE_TYPES)}

def one_hot_gate(name):
    """
    Return one-hot encoding for the given gate name.
    """
    v = [0.0] * len(GATE_TYPES)
    i = GATE_TYPE_IDX.get(name, -1)
    if i >= 0: v[i] = 1.0
    return v

## Quantum Circuits Generators

In [6]:
def generate_variational_ansatz(n_qubits):
    """
    Variational Ansatz for Class A.
    - Uses layers = max(1, n_qubits // 2)
    - Sparse entanglement: Alternate neighboring pairs connected
    """
    layers = max(1, n_qubits // 2)
    qc = QuantumCircuit(n_qubits)
    qubits = list(range(n_qubits))

    for _ in range(layers):
        random.shuffle(qubits)
        for q in qubits:
            theta = random.uniform(0, 2 * np.pi)
            qc.ry(theta, q)

        for i in range(0, n_qubits, 2):
            qc.cx(i, (i + 1) % n_qubits)

    return qc


def generate_qaoa_like(n_qubits):
    """
    QAOA-like Ansatz for Class B.
    - Uses p = max(1, n_qubits // 2)
    - Sparse entanglement: alternate pairs
    """
    p = max(1, n_qubits // 2)
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))

    for _ in range(p):
        gamma = random.uniform(0, 2 * np.pi)
        for i in range(0, n_qubits, 2):
            qc.cx(i, (i + 1) % n_qubits)
            qc.rz(2 * gamma, (i + 1) % n_qubits)
            qc.cx(i, (i + 1) % n_qubits)

        beta = random.uniform(0, 2 * np.pi)
        for q in range(n_qubits):
            qc.rx(2 * beta, q)

    return qc

## Noise Model and Calibration Retrieval

In [7]:
def get_noise_model_and_calib(n_qubits, backend_name="ibm_sherbrooke", noisy=True):
    """
    Get noise model, per-qubit calibration (T1, T2, readout), and gate errors.
    Returns:
        - noise_model
        - per_qubit_calib: List[[T1, T2, readout]]
        - per_gate_errors: Dict[str, float] (e.g. 'cx_0_1', 'rx_0')
    """
    if not noisy:
        per_qubit_calib = [[0.0, 0.0, 0.0] for _ in range(n_qubits)]
        per_gate_errors = {}
        for g in GATE_TYPES:
            if g == "cx":
                for q0 in range(n_qubits):
                    for q1 in range(n_qubits):
                        if q0 != q1:
                            per_gate_errors[f"cx_{q0}_{q1}"] = 0.0
            else:
                for q in range(n_qubits):
                    per_gate_errors[f"{g}_{q}"] = 0.0
        return None, per_qubit_calib, per_gate_errors

    try:
        token = os.getenv("IBM-QUANTUM-THESIS-WORK")
        if not token:
            raise RuntimeError("IBM-QUANTUM-THESIS-WORK token not set.")

        QiskitRuntimeService.save_account(token=token, instance="THESIS-WORK", overwrite=True)
        service = QiskitRuntimeService(channel="ibm_quantum_platform")
        backend = service.backend(backend_name)
        noise_model = NoiseModel.from_backend(backend)
        properties = backend.properties()

        per_qubit_calib = []
        for q in range(n_qubits):
            t1 = properties.t1(q) or 0.0
            t2 = properties.t2(q) or 0.0
            readout_err = properties.readout_error(q) or 0.0
            per_qubit_calib.append([t1/1e5, t2/1e5, readout_err])

        per_gate_errors = {}
        for g in GATE_TYPES:
            if g == "cx":
                for q0 in range(n_qubits):
                    for q1 in range(n_qubits):
                        if q0 != q1:
                            try:
                                err = properties.gate_error("cx", [q0, q1])
                            except Exception:
                                err = 0.0
                            per_gate_errors[f"cx_{q0}_{q1}"] = err
            else:
                for q in range(n_qubits):
                    try:
                        err = properties.gate_error(g, [q])
                    except Exception:
                        err = 0.0
                    per_gate_errors[f"{g}_{q}"] = err

        print(f"Loaded noise errors from {backend_name}")
        return noise_model, per_qubit_calib, per_gate_errors

    except Exception as e:
        print(f"Could not load real hardware noise: {e}")
        per_qubit_calib = [[0.0, 0.0, 0.0] for _ in range(n_qubits)]
        per_gate_errors = {}
        for g in GATE_TYPES:
            if g == "cx":
                for q0 in range(n_qubits):
                    for q1 in range(n_qubits):
                        if q0 != q1:
                            per_gate_errors[f"cx_{q0}_{q1}"] = 0.0
            else:
                for q in range(n_qubits):
                    per_gate_errors[f"{g}_{q}"] = 0.0
        return None, per_qubit_calib, per_gate_errors

In [8]:
# Calibration Snapshot Saving
def save_calibration_snapshot(per_qubit_calib, per_gate_errors, n_qubits, backend_name, root='../datasets'):
    """
    Save calibration snapshot as JSON in datasets/{n_qubit}-qubit/noisy/
    Includes T1, T2, readout and per-gate errors.
    """
    folder = ensure_dataset_dir(n_qubits, "noisy", root=root)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    fname = f"calibration_{n_qubits}q_noisy_{backend_name}_{timestamp}.json"
    path = os.path.join(folder, fname)
    out = {
        "backend": backend_name,
        "timestamp": timestamp,
        "n_qubits": n_qubits,
        "per_qubit_calib": per_qubit_calib,
        "per_gate_errors": per_gate_errors
    }
    with open(path, 'w') as f:
        json.dump(out, f, indent=2)
    print(f"Saved calibration snapshot to {path}")

## Laplacian Eigenvector Encoding

In [9]:
def laplacian_eigenvectors_from_edge_index(edge_index, num_nodes, k=8):
    """
    Compute k non-trivial Laplacian eigenvectors for a graph.
    Returns [num_nodes, k] array. Uses dense fallback or zero padding if needed.
    """
    A = np.zeros((num_nodes, num_nodes))
    if edge_index.shape[1] > 0:
        src, dst = edge_index
        for i, j in zip(src, dst):
            if i != j:
                A[i, j] = 1.0

    D = np.diag(A.sum(axis=1))
    L = D - A

    if num_nodes <= k + 1 or num_nodes <= 100:
        try:
            w, v = np.linalg.eigh(L)
            return v[:, 1:k+1].astype(np.float32)
        except Exception as e:
            print(f"Laplacian dense eig failed: {e}")
            return np.zeros((num_nodes, k), dtype=np.float32)

    try:
        ncv = min(max(2 * (k + 1), 20), num_nodes)
        vals, vecs = eigsh(L, k=k+1, sigma=0.0, which='LM', ncv=ncv)
        return vecs[:, 1:k+1].astype(np.float32)
    except Exception as e:
        print(f"Laplacian eigsh failed: {e}")
        return np.zeros((num_nodes, k), dtype=np.float32)

## Circuit to Graph Conversion (DAG)

In [10]:
def circuit_to_graph(qc, per_qubit_calib, per_gate_errors, n_qubits, class_label, k_lap=8):

    MAX_QUBITS = 6
    num_lap = k_lap
    dag = circuit_to_dag(qc)
    nodes = list(dag.topological_op_nodes())
    if not nodes:
        return None

    node_indices = {node: i for i, node in enumerate(nodes)}
    num_nodes = len(nodes)
    ops = qc.count_ops()
    single_count = sum(ops.get(g, 0) for g in ['rx','ry','rz','h','x','y','z'])
    two_count = ops.get('cx', 0)

    edges, edge_attrs = [], []
    for src, dst, _ in dag.edges():
        if src not in node_indices or dst not in node_indices:
            continue
        i, j = node_indices[src], node_indices[dst]
        dist = abs(i - j) / max(num_nodes - 1, 1)
        src_gate_name = src.name.lower()
        is_cx = 1.0 if src_gate_name == "cx" or dst.name.lower() == "cx" else 0.0
        if src_gate_name == "cx" and len(src.qargs) == 2:
            q0 = qc.find_bit(src.qargs[0]).index
            q1 = qc.find_bit(src.qargs[1]).index
            gate_err_key = f"cx_{q0}_{q1}"
            gate_err = per_gate_errors.get(gate_err_key, 0.0)
        else:
            q = qc.find_bit(src.qargs[0]).index if src.qargs else 0
            gate_err_key = f"{src_gate_name}_{q}"
            gate_err = per_gate_errors.get(gate_err_key, 0.0)
        edges.append([i, j])
        edge_attrs.append([dist, is_cx, gate_err])

    edge_index = torch.tensor(edges, dtype=torch.long).T if edges else torch.empty((2,0), dtype=torch.long)
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float) if edge_attrs else torch.empty((0,3), dtype=torch.float)

    degs = [0] * num_nodes
    for i, j in edges:
        degs[i] += 1
    max_deg = max(degs) or 1

    feats = []
    for idx, node in enumerate(nodes):
        gate_name = node.name.lower()
        qubit = qc.find_bit(node.qargs[0]).index if node.qargs else 0
        angle = float(node.op.params[0]) if hasattr(node.op, 'params') and node.op.params else None
        sincos = [np.sin(angle), np.cos(angle)] if angle is not None else [0.0, 0.0]
        gate_type = one_hot_gate(gate_name)
        qubit_onehot = [1.0 if i == qubit else 0.0 for i in range(MAX_QUBITS)]
        calib = per_qubit_calib[qubit] if per_qubit_calib and qubit < len(per_qubit_calib) else [0.0, 0.0, 0.0]
        if gate_name == "cx" and len(node.qargs) == 2:
            q0 = qc.find_bit(node.qargs[0]).index
            q1 = qc.find_bit(node.qargs[1]).index
            gate_err_key = f"cx_{q0}_{q1}"
            gate_err = per_gate_errors.get(gate_err_key, 0.0)
        else:
            gate_err_key = f"{gate_name}_{qubit}"
            gate_err = per_gate_errors.get(gate_err_key, 0.0)
        norm_layer = idx / max(num_nodes-1, 1)
        deg_norm = degs[idx] / max_deg
        feats.append(sincos + gate_type + qubit_onehot + calib + [gate_err, deg_norm, norm_layer])

    lap_pos = laplacian_eigenvectors_from_edge_index(edge_index.numpy(), num_nodes, k=num_lap)
    if lap_pos.shape[1] < num_lap:
        lap_pos = np.pad(lap_pos, ((0, 0), (0, num_lap - lap_pos.shape[1])), mode='constant')

    x_rows = []
    for i, f in enumerate(feats):
        lap = lap_pos[i] if i < len(lap_pos) else [0.0]*num_lap
        x_rows.append(f + list(lap))
    
    if len(x_rows) == 0 or len(x_rows[0]) != 30:
        return None

    x = torch.tensor(x_rows, dtype=torch.float)

    depth = qc.depth() / 200.0
    cnots = two_count / 100.0
    entangled = 1.0 if two_count > 0 else 0.0
    single_n = single_count / 50.0
    nN = num_nodes
    unique_e = len(edges) / 2
    density = unique_e / (nN*(nN-1)/2) if nN > 1 else 0.0
    avg_calib = np.mean(np.array(per_qubit_calib), axis=0) if per_qubit_calib else [0.0, 0.0, 0.0]
    avg_gate_err = np.mean(list(per_gate_errors.values())) if per_gate_errors else 0.0
    u = torch.tensor([depth, cnots, entangled, single_n, density] + list(avg_calib) + [avg_gate_err], dtype=torch.float)

    pos = torch.arange(num_nodes, dtype=torch.long)
    data = Data(
        x=x, edge_index=edge_index, edge_attr=edge_attr, pos=pos, u=u
    )
    data.class_label = int(class_label)
    return data

## Dataset Generation

In [11]:
def generate_dataset(
    n_qubits,
    circuit_class,
    num_samples,
    noise_type,
    class_params=None,
    backend_name="ibm_sherbrooke",
    shots=1024,
    batch_size=100,
    root='../datasets'
):
    """
    Generate and save dataset for one class (classA/classB), qubit count, and noise setting.
    Stores to datasets/{n_qubit}-qubit/{noisy|noiseless}/{classA|classB}/
    Adds .class_label: 0 (A), 1 (B).
    """
    if circuit_class == "classA":
        gen_func = lambda: generate_variational_ansatz(n_qubits)
    else:
        gen_func = lambda: generate_qaoa_like(n_qubits)

    class_label = 0 if circuit_class == "classA" else 1
    noisy = (noise_type == "noisy")
    noise_model, per_qubit_calib, per_gate_errors = get_noise_model_and_calib(
        n_qubits, backend_name=backend_name, noisy=noisy)
    simulator = AerSimulator()

    data_list = []
    for b in range((num_samples + batch_size - 1) // batch_size):
        bs = min(batch_size, num_samples - b*batch_size)
        before = len(data_list)
        for _ in range(bs):
            qc = gen_func()
            transp = transpile(qc, basis_gates=['u1', 'u2', 'u3', 'cx'], optimization_level=0)
            qc_meas = transp.copy()
            qc_meas.measure_all()

            if noise_model is not None:
                job = simulator.run([qc_meas], shots=shots, noise_model=noise_model)
            else:
                job = simulator.run([qc_meas], shots=shots)
            result = job.result()
            counts = result.get_counts()
            if isinstance(counts, list): counts = counts[0]

            dim = 2 ** n_qubits
            y_np = np.zeros(dim)
            total = sum(counts.values())
            for bits, c in counts.items():
                idx = int(bits.replace(' ', '')[::-1], 2)
                y_np[idx] = c / total
            eps = 1e-6
            y_np = (1 - eps) * y_np + eps / dim
            y = torch.tensor(y_np, dtype=torch.float)

            graph_data = circuit_to_graph(qc, per_qubit_calib, per_gate_errors, n_qubits, class_label, k_lap=8)
            if graph_data is None: continue
            graph_data.y = y
            data_list.append(graph_data)
        generated = len(data_list) - before
        print(f"Batch {b+1} Generated {generated} samples for {n_qubits}q, {noise_type}, {circuit_class}")

    save_dataset(data_list, n_qubits, noise_type, circuit_class, root=root)
    return data_list, (per_qubit_calib, per_gate_errors) if noisy else None

In [12]:
def generate_6q_extrapolation_data(
    num_samples_per_class=500,
    shots=1024,
    backend_name="ibm_sherbrooke",
    overwrite=False,
    root='../datasets'
):
    """
    Generate extrapolation datasets for 6-qubit circuits across:
      - Class A and Class B
      - Noiseless and noisy conditions
    Saves to: ../datasets/6-qubit/{noisy|noiseless}/{classA|classB}/
    """
    n_qubits = 6
    circuit_classes = {"classA": {}, "classB": {}}
    noise_types = ["noiseless", "noisy"]
    batch_size = 100
    result_dict = {}

    # Save calibration snapshot
    if "noisy" in noise_types:
        folder = ensure_dataset_dir(n_qubits, "noisy", circuit_class=None, root=root)
        existing = [f for f in os.listdir(folder) if f.startswith("calibration_")]
        if not existing or overwrite:
            _, per_qubit_calib, per_gate_errors = get_noise_model_and_calib(n_qubits, noisy=True)
            save_calibration_snapshot(per_qubit_calib, per_gate_errors, n_qubits, backend_name, root=root)

    for noise_type in noise_types:
        for circuit_class, class_params in circuit_classes.items():
            folder = ensure_dataset_dir(n_qubits, noise_type, circuit_class, root=root)
            fname = f"dataset_{n_qubits}q_{noise_type}_{circuit_class}.pt"
            save_path = os.path.join(folder, fname)

            print(f"\n6Q Generating {circuit_class}, {noise_type}")
            dataset, _ = generate_dataset(
                n_qubits=n_qubits,
                circuit_class=circuit_class,
                num_samples=num_samples_per_class,
                noise_type=noise_type,
                class_params=class_params,
                backend_name=backend_name,
                shots=shots,
                batch_size=batch_size,
                root=root
            )
            result_dict[(noise_type, circuit_class)] = dataset
    return result_dict

In [13]:
EXTRAP_SAMPLES = 500
SHOTS = 1024
BACKEND = "ibm_sherbrooke"

extrap_data = generate_6q_extrapolation_data(
    num_samples_per_class=EXTRAP_SAMPLES,
    shots=SHOTS,
    backend_name=BACKEND,
    overwrite=False
)

# Summary
print("\n6-Qubit Extrapolation Dataset Summary:")
for (noise_type, circuit_class), data_list in extrap_data.items():
    print(f"  {noise_type}/{circuit_class}: {len(data_list)} samples")

Loaded noise errors from ibm_sherbrooke
Saved calibration snapshot to ../datasets\6-qubit\noisy\calibration_6q_noisy_ibm_sherbrooke_20250716_200853.json

6Q Generating classA, noiseless
Batch 1 Generated 100 samples for 6q, noiseless, classA
Batch 2 Generated 100 samples for 6q, noiseless, classA
Batch 3 Generated 100 samples for 6q, noiseless, classA
Batch 4 Generated 100 samples for 6q, noiseless, classA
Batch 5 Generated 100 samples for 6q, noiseless, classA
Saved dataset (500 samples) to ../datasets\6-qubit\noiseless\classA\dataset_6q_noiseless_classA.pt

6Q Generating classB, noiseless
Batch 1 Generated 100 samples for 6q, noiseless, classB
Batch 2 Generated 100 samples for 6q, noiseless, classB
Batch 3 Generated 100 samples for 6q, noiseless, classB
Batch 4 Generated 100 samples for 6q, noiseless, classB
Batch 5 Generated 100 samples for 6q, noiseless, classB
Saved dataset (500 samples) to ../datasets\6-qubit\noiseless\classB\dataset_6q_noiseless_classB.pt

6Q Generating classA, 